In [10]:
library(dplyr)
library(purrr)
library(tidyr)
library(ggplot2)

In [11]:
load('IndigoData.dat')
data_ps <- data %>% filter(event_state == "PRINT_STATE")

Assuming the that the population we want to find a sample size to represent is the number of events we do have now, how many events chosen at random we have to have given an expected margin of error (E) and a confidence level (CL)? We can calculate this using this equation:

$$n=  \frac{\frac{z^2 p(1-p)}{e^2}}{1+\left(\frac{z^2 p(1-p)}{e^2 N}\right)}$$


In [75]:
E <- c(0.05, 0.01, 0.001, 0.05, 0.01, 0.001)
CL <- c(0.95, 0.95, 0.95, 0.99, 0.99, 0.99)
p <- 0.50 
sample.size.prop <- function(E, CL, p, N) {
    Z <- qnorm((1 - CL)/2, 0, 1, lower.tail = F) #Z-Score for the confidence level
    partial <- (Z^2 * p * (1 - p)) / E^2
    partial <- partial / (1 + partial * (1/N))
    ceiling(partial)
}

sample_sizes <- data.frame(MarginError = E, ConfidenceLevel = CL) %>%
                mutate( SampleSize.All = mapply(sample.size, E, CL, MoreArgs = list(p = p, N = nrow(data_ps))),
                        SampleSize.HP_Indigo_7000 = mapply(sample.size.prop, E, CL, MoreArgs = list(p = p, N = data_ps %>%
                                                                                                 filter(Product_Line == "HP Indigo 7000") %>%
                                                                                                 nrow())),
                       SampleSize.HP_Indigo_7500 = mapply(sample.size.prop, E, CL, MoreArgs = list(p = p, N = data_ps %>%
                                                                                                 filter(Product_Line == "HP Indigo 7500") %>%
                                                                                                 nrow())),
                       SampleSize.HP_Indigo_7600 = mapply(sample.size.prop, E, CL, MoreArgs = list(p = p, N = data_ps %>%
                                                                                                 filter(Product_Line == "HP Indigo 7600") %>%
                                                                                                 nrow())))
sample_sizes




,MarginError,ConfidenceLevel,SampleSize.All,SampleSize.HP_Indigo_7000,SampleSize.HP_Indigo_7500,SampleSize.HP_Indigo_7600
1,0.05,0.95,384.00,382.00,382.00,383.00
2,0.01,0.95,9172.00,8072.00,8385.00,8650.00
3,0.001,0.950,168038.000,48060.000,61788.000,79814.000
4,0.05,0.99,662.00,655.00,657.00,659.00
5,0.01,0.99,15339.00,12492.00,13258.00,13933.00
6,0.001,0.990,181401.000,49094.000,63508.000,82708.000


How many presses have a number of events that satisfy the suggestion?

In [72]:
print("Population is all PRINT_STATE events")

summ_all <- data_ps %>% group_by(event_press) %>%
  summarise(events = n()) %>% 
  summarise(E0.05.CL0.95 = round(sum(events >= 384) / n(), 2),
            E0.01.CL0.95 =  round(sum(events >= 9172.00) / n(), 2),
            E0.001.CL0.95 =  round(sum(events >= 168038.000) / n(), 2),
            E0.05.CL0.99 =  round(sum(events >= 662.00) / n(), 2),
            E0.01.CL0.99 =  round(sum(events >= 15339.00) / n(), 2),
            E0.001.CL0.99 =  round(sum(events >= 181401.000) / n(), 2),
            PressRatio = 1,
            Presses = n())
summ_all

print("Population is all PRINT_STATE events where Product_Line is HP Indigo 7000")
data_ps %>% group_by(event_press) %>%
            filter(Product_Line == "HP Indigo 7000") %>%
  summarise(events = n())%>% 
  summarise(E0.05.CL0.95 = round(sum(events >= 382.00) / n(), 2),
            E0.01.CL0.95 =  round(sum(events >= 8072.00) / n(), 2),
            E0.001.CL0.95 =  round(sum(events >= 48060.000) / n(), 2),
            E0.05.CL0.99 =  round(sum(events >= 655.00) / n(), 2),
            E0.01.CL0.99 =  round(sum(events >= 12492.00) / n(), 2),
            E0.001.CL0.99 =  round(sum(events >= 49094.000) / n(), 2),
            PressRatio = round(n() / summ_all$Presses, 2),
            Presses = n())

print("Population is all PRINT_STATE events where Product_Line is HP Indigo 7500")
data_ps %>% group_by(event_press) %>%
            filter(Product_Line == "HP Indigo 7500") %>%
  summarise(events = n())%>% 
  summarise(E0.05.CL0.95 = round(sum(events >= 382) / n(), 2),
            E0.01.CL0.95 =  round(sum(events >= 8385.00) / n(), 2),
            E0.001.CL0.95 =  round(sum(events >= 61788.000) / n(), 2),
            E0.05.CL0.99 =  round(sum(events >= 657.00) / n(), 2),
            E0.01.CL0.99 =  round(sum(events >= 13258.00) / n(), 2),
            E0.001.CL0.99 =  round(sum(events >= 63508.000) / n(), 2),
            PressRatio = round(n() / summ_all$Presses, 2),
            Presses = n())

print("Population is all PRINT_STATE events where Product_Line is HP Indigo 7600")
data_ps %>% group_by(event_press) %>%
            filter(Product_Line == "HP Indigo 7600") %>%
  summarise(events = n())%>% 
  summarise(E0.05.CL0.95 = round(sum(events >= 383) / n(), 2),
            E0.01.CL0.95 =  round(sum(events >= 8650.00) / n(), 2),
            E0.001.CL0.95 =  round(sum(events >= 79814.000) / n(), 2),
            E0.05.CL0.99 =  round(sum(events >= 659.00) / n(), 2),
            E0.01.CL0.99 =  round(sum(events >= 13933.00) / n(), 2),
            E0.001.CL0.99 =  round(sum(events >= 82708.000) / n(), 2),
            PressRatio = round(n() / summ_all$Presses, 2),
            Presses = n())

[1] "Population is all PRINT_STATE events"


,E0.05.CL0.95,E0.01.CL0.95,E0.001.CL0.95,E0.05.CL0.99,E0.01.CL0.99,E0.001.CL0.99,PressRatio,Presses
1,0.20,0.00,0.00,0.08,0.00,0.00,1.00,851.00


[1] "Population is all PRINT_STATE events where Product_Line is HP Indigo 7000"


,E0.05.CL0.95,E0.01.CL0.95,E0.001.CL0.95,E0.05.CL0.99,E0.01.CL0.99,E0.001.CL0.99,PressRatio,Presses
1,0.16,0.00,0.00,0.08,0.00,0.00,0.27,228.00


[1] "Population is all PRINT_STATE events where Product_Line is HP Indigo 7500"


,E0.05.CL0.95,E0.01.CL0.95,E0.001.CL0.95,E0.05.CL0.99,E0.01.CL0.99,E0.001.CL0.99,PressRatio,Presses
1,0.22,0.00,0.00,0.11,0.00,0.00,0.29,246.00


[1] "Population is all PRINT_STATE events where Product_Line is HP Indigo 7600"


,E0.05.CL0.95,E0.01.CL0.95,E0.001.CL0.95,E0.05.CL0.99,E0.01.CL0.99,E0.001.CL0.99,PressRatio,Presses
1,0.21,0.00,0.00,0.07,0.00,0.00,0.44,377.00


According to the above calculation we could only satisfy the E 0.05, CL 0.95 with about 20% of the printers, and with a margin of error 1% about 8% of the printers.

It's not much better if we take the population we want to describe as the numeber of events for each product line.

These calculations assume a 50% normal distribution to estimate these values. Based on the [previous investigation](http://15.8.169.215:8888/notebooks/Normal%20Distribution%20critical%20values%20for%20features.ipynb#), only some of the values somewhat fit a normal distribution.


### Using standard deviation estimating sample size
Another way to estimate sample size is based on the standard deviation instead of the expected proportion:

$$n=  \frac{\frac{z^2\sigma^2}{e^2}}{1+\left(\frac{z^2\sigma^2}{e^2 N}\right) }$$

In [73]:
sample.size.sigma <- function(E, CL, sigma, N) {
    if(is.na(sigma) || is.na(N)) {
        return(NA)
    }
    Z <- qnorm((1 - CL)/2, 0, 1, lower.tail = F) #Z-Score for the confidence level
    partial <- (Z^2 * sigma^2) / E^2
    partial <- partial / (1 + partial * (1/N))
    ceiling(partial)
}

summ <- data_ps %>% 
  summarise_each(funs = funs(mean(., na.rm = T), sd(., na.rm = T), n()), 
                 PIP_Temperature, IO_temperature, vessel_flow, IO_dirtiness, CS_Voltage, IO_Conductivity, Delta_Pressure) %>%
  mutate(Product_Line = "All") %>% 
  select(Product_Line, ends_with("mean"), ends_with("sd"), ends_with("n"))

summ_pl <- data_ps %>% 
  group_by(Product_Line) %>%
  summarise_each(funs = funs(mean(., na.rm = T), sd(., na.rm = T), n()), 
                 PIP_Temperature, IO_temperature, vessel_flow, IO_dirtiness, CS_Voltage, IO_Conductivity, Delta_Pressure)

summ <- rbind(summ, summ_pl)
sample_size_sigma <- summ %>% gather(key = Feature, value = Value, -Product_Line) %>%
         mutate(Feature = gsub("_mean", ".mean", Feature),
                Feature = gsub("_sd", ".sd", Feature),
                Feature = gsub("_n", ".n", Feature)) %>%
         separate(Feature, c("Feature", "Metric"), sep = "[.]") %>%
         spread(key = Metric, value = Value) %>%
         mutate(
            SS_E0.05CL0.95 = sample.size.sigma(0.05, 0.95, sd, n),
            SS_E0.01CL0.95 = sample.size.sigma(0.01, 0.95, sd, n),
            SS_E0.001CL0.95 = sample.size.sigma(0.001, 0.95, sd, n),
            SS_E0.05CL0.99 = sample.size.sigma(0.05, 0.99, sd, n),
            SS_E0.01CL0.99 = sample.size.sigma(0.01, 0.99, sd, n),
            SS_E0.001CL0.99 = sample.size.sigma(0.001, 0.99, sd, n)
         ) %>%
         select(Product_Line, Feature, n, starts_with("SS"))
sample_size_sigma


,Product_Line,Feature,n,SS_E0.05CL0.95,SS_E0.01CL0.95,SS_E0.001CL0.95,SS_E0.05CL0.99,SS_E0.01CL0.99,SS_E0.001CL0.99
1,All,CS_Voltage,203675,203296,203660,203675,203456,203667,203675
2,All,Delta_Pressure,203675,170645,202111,203660,183150,202767,203666
3,All,IO_Conductivity,203675,3317,59613,198870,5661,84894,200865
4,All,IO_dirtiness,203675,203498,203668,203675,203573,203671,203675
5,All,IO_temperature,203675,185846,202897,203668,192957,203224,203671
6,All,PIP_Temperature,203675,164936,201780,203656,179294,202574,203664
7,All,vessel_flow,203675,203634,203674,203675,203652,203675,203675
8,HP Indigo 7000,CS_Voltage,50591,50573,50591,50591,50581,50591,50591
9,HP Indigo 7000,Delta_Pressure,50591,46728,50425,50590,48280,50495,50591
10,HP Indigo 7000,IO_Conductivity,50591,NA,NA,NA,NA,NA,NA


Including the standard deviations in the calculations seems to worsen the case, as the suggested sample sizes are almost the same size of the population. This is due to the high standard deviation values most of the features have.

### Using standard deviation estimating sample size
Yet another way to estimate sample size based on the standard deviation:

$$n=  \frac{\frac{z^2\sigma^2N}{e^2}}{(N - 1)+\left(\frac{z^2\sigma^2}{e^2}\right) }$$

In [80]:
#fpc means Finite Population Correction
sample.size.sigma.fpc <- function(E, CL, sigma, N) {
    if(is.na(sigma) || is.na(N)) {
        return(NA)
    }
    Z <- qnorm((1 - CL)/2, 0, 1, lower.tail = F) #Z-Score for the confidence level
    partial <- (Z^2 * sigma^2) / E^2
    partial <- (partial * N) / (N-1 + partial)
    ceiling(partial)
}

summ <- data_ps %>% 
  summarise_each(funs = funs(mean(., na.rm = T), sd(., na.rm = T), n()), 
                 PIP_Temperature, IO_temperature, vessel_flow, IO_dirtiness, CS_Voltage, IO_Conductivity, Delta_Pressure) %>%
  mutate(Product_Line = "All") %>% 
  select(Product_Line, ends_with("mean"), ends_with("sd"), ends_with("n"))

summ_pl <- data_ps %>% 
  group_by(Product_Line) %>%
  summarise_each(funs = funs(mean(., na.rm = T), sd(., na.rm = T), n()), 
                 PIP_Temperature, IO_temperature, vessel_flow, IO_dirtiness, CS_Voltage, IO_Conductivity, Delta_Pressure)

summ <- rbind(summ, summ_pl)
sample_size_sigma_fpc <- summ %>% gather(key = Feature, value = Value, -Product_Line) %>%
         mutate(Feature = gsub("_mean", ".mean", Feature),
                Feature = gsub("_sd", ".sd", Feature),
                Feature = gsub("_n", ".n", Feature)) %>%
         separate(Feature, c("Feature", "Metric"), sep = "[.]") %>%
         spread(key = Metric, value = Value) %>%
         mutate(
            SS_E0.05CL0.95 = sample.size.sigma.fpc(0.05, 0.95, sd, n),
            SS_E0.01CL0.95 = sample.size.sigma.fpc(0.01, 0.95, sd, n),
            SS_E0.001CL0.95 = sample.size.sigma.fpc(0.001, 0.95, sd, n),
            SS_E0.05CL0.99 = sample.size.sigma.fpc(0.05, 0.99, sd, n),
            SS_E0.01CL0.99 = sample.size.sigma.fpc(0.01, 0.99, sd, n),
            SS_E0.001CL0.99 = sample.size.sigma.fpc(0.001, 0.99, sd, n)
         ) %>%
         select(Product_Line, Feature, n, starts_with("SS"))
sample_size_sigma_fpc

,Product_Line,Feature,n,SS_E0.05CL0.95,SS_E0.01CL0.95,SS_E0.001CL0.95,SS_E0.05CL0.99,SS_E0.01CL0.99,SS_E0.001CL0.99
1,All,CS_Voltage,203675,203296,203660,203675,203456,203667,203675
2,All,Delta_Pressure,203675,170645,202111,203660,183150,202767,203666
3,All,IO_Conductivity,203675,3317,59614,198870,5661,84895,200865
4,All,IO_dirtiness,203675,203498,203668,203675,203573,203671,203675
5,All,IO_temperature,203675,185846,202897,203668,192957,203224,203671
6,All,PIP_Temperature,203675,164936,201780,203656,179294,202574,203664
7,All,vessel_flow,203675,203634,203674,203675,203652,203675,203675
8,HP Indigo 7000,CS_Voltage,50591,50573,50591,50591,50581,50591,50591
9,HP Indigo 7000,Delta_Pressure,50591,46728,50425,50590,48280,50495,50591
10,HP Indigo 7000,IO_Conductivity,50591,NA,NA,NA,NA,NA,NA


Using the alternate formula correcting for finite population the suggested sample sizes are still too large. A lot larger than suggested by the 50% normal distribution that dramatically reduces the number of presses that qualifies having a sufficient number of events to describe the population.


### Scaling variables to reduce standard deviation values

If we scale the sensor values we can have a standard deviation with a lower magnitude, which would influnce the equation.

In [176]:
#3rd Quartile + 1.5IQR means we're ignoring extreme outliers to compute max
#this is used on boxplots in the same manner.

quantile.max <- function(feature) {
    quantile(feature, 0.75, na.rm = T) + (1.5 * IQR(feature, na.rm = T))
}

#Adding a small value to the denominator allows us to account for when the max is 0.

summ <- data_ps %>% 
  mutate(
      PIP_Temperature = PIP_Temperature/(quantile.max(PIP_Temperature) + 0.00000001),
      IO_temperature = IO_temperature/(quantile.max(IO_temperature) + 0.00000001),
      vessel_flow = vessel_flow/(quantile.max(vessel_flow) + 0.00000001),
      IO_dirtiness = IO_dirtiness/(quantile.max(IO_dirtiness) + 0.00000001),
      CS_Voltage = CS_Voltage/(quantile.max(CS_Voltage) + 0.00000001),
      IO_Conductivity = IO_Conductivity/(quantile.max(IO_Conductivity) + 0.00000001),
      Delta_Pressure = Delta_Pressure/(quantile.max(Delta_Pressure) + 0.00000001)
  ) %>%
  summarise_each(funs = funs(mean(., na.rm = T), sd(., na.rm = T), n()), 
                 PIP_Temperature, IO_temperature, vessel_flow, IO_dirtiness, CS_Voltage, IO_Conductivity, Delta_Pressure) %>%
  mutate(Product_Line = "All") %>% 
  select(Product_Line, ends_with("mean"), ends_with("sd"), ends_with("n"))

summ_pl <- data_ps %>% 
  mutate(
      PIP_Temperature = PIP_Temperature/(quantile.max(PIP_Temperature) + 0.00000001),
      IO_temperature = IO_temperature/(quantile.max(IO_temperature) + 0.00000001),
      vessel_flow = vessel_flow/(quantile.max(vessel_flow) + 0.00000001),
      IO_dirtiness = IO_dirtiness/(quantile.max(IO_dirtiness) + 0.00000001),
      CS_Voltage = CS_Voltage/(quantile.max(CS_Voltage) + 0.00000001),
      IO_Conductivity = IO_Conductivity/(quantile.max(IO_Conductivity) + 0.00000001),
      Delta_Pressure = Delta_Pressure/(quantile.max(Delta_Pressure) + 0.00000001)
  ) %>%
  group_by(Product_Line) %>%
  summarise_each(funs = funs(mean(., na.rm = T), sd(., na.rm = T), n()), 
                 PIP_Temperature, IO_temperature, vessel_flow, IO_dirtiness, CS_Voltage, IO_Conductivity, Delta_Pressure)

summ <- rbind(summ, summ_pl)
sample_size_sigma_scaled <- summ %>% gather(key = Feature, value = Value, -Product_Line) %>%
         mutate(Feature = gsub("_mean", ".mean", Feature),
                Feature = gsub("_sd", ".sd", Feature),
                Feature = gsub("_n", ".n", Feature)) %>%
         separate(Feature, c("Feature", "Metric"), sep = "[.]") %>%
         spread(key = Metric, value = Value) %>%
         mutate(
            SS_E0.05CL0.95 = sample.size.sigma(0.05, 0.95, sd, n),
            SS_E0.01CL0.95 = sample.size.sigma(0.01, 0.95, sd, n),
            SS_E0.001CL0.95 = sample.size.sigma(0.001, 0.95, sd, n),
            SS_E0.05CL0.99 = sample.size.sigma(0.05, 0.99, sd, n),
            SS_E0.01CL0.99 = sample.size.sigma(0.01, 0.99, sd, n),
            SS_E0.001CL0.99 = sample.size.sigma(0.001, 0.99, sd, n)
         ) %>%
         select(Product_Line, Feature, n, starts_with("SS"))
sample_size_sigma_scaled


,Product_Line,Feature,n,SS_E0.05CL0.95,SS_E0.01CL0.95,SS_E0.001CL0.95,SS_E0.05CL0.99,SS_E0.01CL0.99,SS_E0.001CL0.99
1,All,CS_Voltage,203675,15,354,30171,25,610,47044
2,All,Delta_Pressure,203675,203676,203676,203675,203676,203675,203676
3,All,IO_Conductivity,203675,53,1309,79978,91,2250,107454
4,All,IO_dirtiness,203675,43,1049,69469,73,1805,96141
5,All,IO_temperature,203675,18,432,35643,30,744,54611
6,All,PIP_Temperature,203675,7,173,15946,12,299,26057
7,All,vessel_flow,203675,42,1044,69254,73,1797,95902
8,HP Indigo 7000,CS_Voltage,50591,19,456,24077,32,782,30893
9,HP Indigo 7000,Delta_Pressure,50591,50592,50592,50591,50591,50592,50591
10,HP Indigo 7000,IO_Conductivity,50591,NA,NA,NA,NA,NA,NA


In [177]:
summ <- data_ps %>% 
  mutate(
      PIP_Temperature = PIP_Temperature/(quantile.max(PIP_Temperature) + 0.00000001),
      IO_temperature = IO_temperature/(quantile.max(IO_temperature) + 0.00000001),
      vessel_flow = vessel_flow/(quantile.max(vessel_flow) + 0.00000001),
      IO_dirtiness = IO_dirtiness/(quantile.max(IO_dirtiness) + 0.00000001),
      CS_Voltage = CS_Voltage/(quantile.max(CS_Voltage) + 0.00000001),
      IO_Conductivity = IO_Conductivity/(quantile.max(IO_Conductivity) + 0.00000001),
      Delta_Pressure = Delta_Pressure/(quantile.max(Delta_Pressure) + 0.00000001)
  ) %>%
  summarise_each(funs = funs(mean(., na.rm = T), sd(., na.rm = T), n()), 
                 PIP_Temperature, IO_temperature, vessel_flow, IO_dirtiness, CS_Voltage, IO_Conductivity, Delta_Pressure) %>%
  mutate(Product_Line = "All") %>% 
  select(Product_Line, ends_with("mean"), ends_with("sd"), ends_with("n"))

summ_pl <- data_ps %>% 
  mutate(
      PIP_Temperature = PIP_Temperature/(quantile.max(PIP_Temperature) + 0.00000001),
      IO_temperature = IO_temperature/(quantile.max(IO_temperature) + 0.00000001),
      vessel_flow = vessel_flow/(quantile.max(vessel_flow) + 0.00000001),
      IO_dirtiness = IO_dirtiness/(quantile.max(IO_dirtiness) + 0.00000001),
      CS_Voltage = CS_Voltage/(quantile.max(CS_Voltage) + 0.00000001),
      IO_Conductivity = IO_Conductivity/(quantile.max(IO_Conductivity) + 0.00000001),
      Delta_Pressure = Delta_Pressure/(quantile.max(Delta_Pressure) + 0.00000001)
  ) %>%
  group_by(Product_Line) %>%
  summarise_each(funs = funs(mean(., na.rm = T), sd(., na.rm = T), n()), 
                 PIP_Temperature, IO_temperature, vessel_flow, IO_dirtiness, CS_Voltage, IO_Conductivity, Delta_Pressure)

summ <- rbind(summ, summ_pl)
sample_size_sigma_fpc_scaled <- summ %>% gather(key = Feature, value = Value, -Product_Line) %>%
         mutate(Feature = gsub("_mean", ".mean", Feature),
                Feature = gsub("_sd", ".sd", Feature),
                Feature = gsub("_n", ".n", Feature)) %>%
         separate(Feature, c("Feature", "Metric"), sep = "[.]") %>%
         spread(key = Metric, value = Value) %>%
         mutate(
            SS_E0.05CL0.95 = sample.size.sigma.fpc(0.05, 0.95, sd, n),
            SS_E0.01CL0.95 = sample.size.sigma.fpc(0.01, 0.95, sd, n),
            SS_E0.001CL0.95 = sample.size.sigma.fpc(0.001, 0.95, sd, n),
            SS_E0.05CL0.99 = sample.size.sigma.fpc(0.05, 0.99, sd, n),
            SS_E0.01CL0.99 = sample.size.sigma.fpc(0.01, 0.99, sd, n),
            SS_E0.001CL0.99 = sample.size.sigma.fpc(0.001, 0.99, sd, n)
         ) %>%
         select(Product_Line, Feature, n, starts_with("SS"))
sample_size_sigma_fpc_scaled

,Product_Line,Feature,n,SS_E0.05CL0.95,SS_E0.01CL0.95,SS_E0.001CL0.95,SS_E0.05CL0.99,SS_E0.01CL0.99,SS_E0.001CL0.99
1,All,CS_Voltage,203675,15,354,30171,25,610,47044
2,All,Delta_Pressure,203675,203675,203675,203675,203675,203676,203675
3,All,IO_Conductivity,203675,53,1309,79979,91,2250,107454
4,All,IO_dirtiness,203675,43,1049,69469,73,1805,96141
5,All,IO_temperature,203675,18,432,35643,30,744,54612
6,All,PIP_Temperature,203675,7,173,15946,12,299,26058
7,All,vessel_flow,203675,42,1044,69255,73,1797,95902
8,HP Indigo 7000,CS_Voltage,50591,19,456,24077,32,782,30894
9,HP Indigo 7000,Delta_Pressure,50591,50592,50591,50591,50591,50591,50591
10,HP Indigo 7000,IO_Conductivity,50591,NA,NA,NA,NA,NA,NA


In [178]:
get.sample.size <- function(df, ProductLine, Feature, Type) {
  ss <- (df %>%
     filter_(paste0("Product_Line == \"", ProductLine,"\""), paste0("Feature == \"", Feature,"\"")) %>%
     select_(paste(Type)))[[1]]
  ss
}

sum.events <- function(events, ...) {
  sample_size <- get.sample.size(...)
  if(is.na(sample_size)) {
    return(NA)
  }
  sum(events >= sample_size)
}

summ_all <- data_ps %>% group_by(event_press) %>%
  summarise(
      events = n(),
      PIP_Temperature.events = sum(!is.na(PIP_Temperature)),
      IO_temperature.events = sum(!is.na(IO_temperature)),
      vessel_flow.events = sum(!is.na(vessel_flow)),
      CS_Voltage.events = sum(!is.na(CS_Voltage)),
      IO_dirtiness.events = sum(!is.na(IO_dirtiness)),
      IO_Conductivity.events = sum(!is.na(IO_Conductivity)),
      Delta_Pressure.events = sum(!is.na(Delta_Pressure)),
      Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) 

feature <- "PIP_Temperature"

PIP_Temperature.All <- summ_all %>%
  summarise(
    E0.05.CL0.95 = (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, "All", feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, "All", feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, "All", feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, "All", feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, "All", feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, "All", feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(PIP_Temperature.events > 0) / n()) %>% round(2),
    Presses = n()
  ) %>%
  mutate_(
    Product_Line = "\"All\"",
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7000"
PIP_Temperature.HP7000 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(PIP_Temperature.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7600"
PIP_Temperature.HP7500 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(PIP_Temperature.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7600"
PIP_Temperature.HP7600 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(PIP_Temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(PIP_Temperature.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

PIP_Temperature.prop <- rbind(PIP_Temperature.All, PIP_Temperature.HP7000, PIP_Temperature.HP7500, PIP_Temperature.HP7600)

In [179]:
feature <- "IO_temperature"

IO_temperature.All <- summ_all %>%
  summarise(
    E0.05.CL0.95 = (sum.events(IO_temperature.events, sample_size_sigma_scaled, "All", feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(IO_temperature.events, sample_size_sigma_scaled, "All", feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(IO_temperature.events, sample_size_sigma_scaled, "All", feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(IO_temperature.events, sample_size_sigma_scaled, "All", feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(IO_temperature.events, sample_size_sigma_scaled, "All", feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(IO_temperature.events, sample_size_sigma_scaled, "All", feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(IO_temperature.events > 0) / n()) %>% round(2),
    Presses = n()
  ) %>%
  mutate_(
    Product_Line = "\"All\"",
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7000"
IO_temperature.HP7000 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(IO_temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(IO_temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(IO_temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(IO_temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(IO_temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(IO_temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(IO_temperature.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7600"
IO_temperature.HP7500 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(IO_temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(IO_temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(IO_temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(IO_temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(IO_temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(IO_temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(IO_temperature.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7600"
IO_temperature.HP7600 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(IO_temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(IO_temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(IO_temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(IO_temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(IO_temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(IO_temperature.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(IO_temperature.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

IO_temperature.prop <- rbind(IO_temperature.All, IO_temperature.HP7000, IO_temperature.HP7500, IO_temperature.HP7600)

In [180]:
feature <- "vessel_flow"

vessel_flow.All <- summ_all %>%
  summarise(
    E0.05.CL0.95 = (sum.events(vessel_flow.events, sample_size_sigma_scaled, "All", feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(vessel_flow.events, sample_size_sigma_scaled, "All", feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(vessel_flow.events, sample_size_sigma_scaled, "All", feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(vessel_flow.events, sample_size_sigma_scaled, "All", feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(vessel_flow.events, sample_size_sigma_scaled, "All", feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(vessel_flow.events, sample_size_sigma_scaled, "All", feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(vessel_flow.events > 0) / n()) %>% round(2),
    Presses = n()
  ) %>%
  mutate_(
    Product_Line = "\"All\"",
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7000"
vessel_flow.HP7000 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(vessel_flow.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(vessel_flow.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(vessel_flow.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(vessel_flow.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(vessel_flow.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(vessel_flow.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(vessel_flow.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7600"
vessel_flow.HP7500 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(vessel_flow.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(vessel_flow.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(vessel_flow.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(vessel_flow.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(vessel_flow.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(vessel_flow.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(vessel_flow.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7600"
vessel_flow.HP7600 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(vessel_flow.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(vessel_flow.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(vessel_flow.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(vessel_flow.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(vessel_flow.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(vessel_flow.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(vessel_flow.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

vessel_flow.prop <- rbind(vessel_flow.All, vessel_flow.HP7000, vessel_flow.HP7500, vessel_flow.HP7600)

In [181]:
feature <- "CS_Voltage"

CS_Voltage.All <- summ_all %>%
  summarise(
    E0.05.CL0.95 = (sum.events(CS_Voltage.events, sample_size_sigma_scaled, "All", feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(CS_Voltage.events, sample_size_sigma_scaled, "All", feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(CS_Voltage.events, sample_size_sigma_scaled, "All", feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(CS_Voltage.events, sample_size_sigma_scaled, "All", feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(CS_Voltage.events, sample_size_sigma_scaled, "All", feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(CS_Voltage.events, sample_size_sigma_scaled, "All", feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(CS_Voltage.events > 0) / n()) %>% round(2),
    Presses = n()
  ) %>%
  mutate_(
    Product_Line = "\"All\"",
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7000"
CS_Voltage.HP7000 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(CS_Voltage.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(CS_Voltage.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(CS_Voltage.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(CS_Voltage.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(CS_Voltage.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(CS_Voltage.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(CS_Voltage.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7600"
CS_Voltage.HP7500 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(CS_Voltage.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(CS_Voltage.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(CS_Voltage.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(CS_Voltage.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(CS_Voltage.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(CS_Voltage.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(CS_Voltage.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7600"
CS_Voltage.HP7600 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(CS_Voltage.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(CS_Voltage.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(CS_Voltage.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(CS_Voltage.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(CS_Voltage.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(CS_Voltage.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(CS_Voltage.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

CS_Voltage.prop <- rbind(CS_Voltage.All, CS_Voltage.HP7000, CS_Voltage.HP7500, CS_Voltage.HP7600)

In [182]:
feature <- "IO_dirtiness"

IO_dirtiness.All <- summ_all %>%
  summarise(
    E0.05.CL0.95 = (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, "All", feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, "All", feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, "All", feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, "All", feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, "All", feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, "All", feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(IO_dirtiness.events > 0) / n()) %>% round(2),
    Presses = n()
  ) %>%
  mutate_(
    Product_Line = "\"All\"",
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7000"
IO_dirtiness.HP7000 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(IO_dirtiness.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7600"
IO_dirtiness.HP7500 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(IO_dirtiness.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7600"
IO_dirtiness.HP7600 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(IO_dirtiness.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(IO_dirtiness.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

IO_dirtiness.prop <- rbind(IO_dirtiness.All, IO_dirtiness.HP7000, IO_dirtiness.HP7500, IO_dirtiness.HP7600)

In [183]:
feature <- "IO_Conductivity"

IO_Conductivity.All <- summ_all %>%
  summarise(
    E0.05.CL0.95 = (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, "All", feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, "All", feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, "All", feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, "All", feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, "All", feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, "All", feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(IO_Conductivity.events > 0) / n()) %>% round(2),
    Presses = n()
  ) %>%
  mutate_(
    Product_Line = "\"All\"",
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7000"
IO_Conductivity.HP7000 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(IO_Conductivity.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7600"
IO_Conductivity.HP7500 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(IO_Conductivity.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7600"
IO_Conductivity.HP7600 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(IO_Conductivity.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(IO_Conductivity.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

IO_Conductivity.prop <- rbind(IO_Conductivity.All, IO_Conductivity.HP7000, IO_Conductivity.HP7500, IO_Conductivity.HP7600)

In [184]:
feature <- "Delta_Pressure"

Delta_Pressure.All <- summ_all %>%
  summarise(
    E0.05.CL0.95 = (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, "All", feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, "All", feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, "All", feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, "All", feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, "All", feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, "All", feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(Delta_Pressure.events > 0) / n()) %>% round(2),
    Presses = n()
  ) %>%
  mutate_(
    Product_Line = "\"All\"",
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7000"
Delta_Pressure.HP7000 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(Delta_Pressure.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7600"
Delta_Pressure.HP7500 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(Delta_Pressure.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

product_line <- "HP Indigo 7600"
Delta_Pressure.HP7600 <- summ_all %>%
  filter_(paste0("Product_Line == \"",product_line, "\"")) %>%
  summarise(
    E0.05.CL0.95 = (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.95") / n()) %>% round(2),
    E0.01.CL0.95 =  (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.95") / n()) %>% round(2),
    E0.001.CL0.95 =  (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.95") / n()) %>% round(2),
    E0.05.CL0.99 = (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.05CL0.99") / n()) %>% round(2),
    E0.01.CL0.99 =  (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.01CL0.99") / n()) %>% round(2),
    E0.001.CL0.99 =  (sum.events(Delta_Pressure.events, sample_size_sigma_scaled, product_line, feature, "SS_E0.001CL0.99") / n()) %>% round(2),
    PressRatio = (sum(Delta_Pressure.events > 0) / n()) %>% round(2),
    Presses = n(),
    Product_Line = unique(Product_Line) %>% paste0(collapse = "|")
  ) %>%
  mutate_(
    Feature = paste0("\"",feature,"\"")
  ) %>%
  select(Product_Line, Feature, starts_with("E"), PressRatio, Presses)

Delta_Pressure.prop <- rbind(Delta_Pressure.All, Delta_Pressure.HP7000, Delta_Pressure.HP7500, Delta_Pressure.HP7600)

In [185]:
rbind(PIP_Temperature.prop, IO_temperature.prop, vessel_flow.prop, CS_Voltage.prop, IO_dirtiness.prop, IO_Conductivity.prop, Delta_Pressure.prop) %>%
arrange(Product_Line, Feature)

,Product_Line,Feature,E0.05.CL0.95,E0.01.CL0.95,E0.001.CL0.95,E0.05.CL0.99,E0.01.CL0.99,E0.001.CL0.99,PressRatio,Presses
1,All,CS_Voltage,0.89,0.22,0,0.85,0.1,0,1,851
2,All,Delta_Pressure,0,0,0,0,0,0,1,851
3,All,IO_Conductivity,0.37,0.01,0,0.29,0,0,0.52,851
4,All,IO_dirtiness,0.76,0.03,0,0.63,0,0,1,851
5,All,IO_temperature,0.87,0.17,0,0.83,0.06,0,1,851
6,All,PIP_Temperature,0.95,0.4,0,0.9,0.26,0,1,851
7,All,vessel_flow,0.7,0.02,0,0.58,0,0,0.94,851
8,HP Indigo 7000,CS_Voltage,0.88,0.12,0,0.81,0.06,0,1,228
9,HP Indigo 7000,Delta_Pressure,0,0,0,0,0,0,1,228
10,HP Indigo 7000,IO_Conductivity,NA,NA,NA,NA,NA,NA,0,228


Doing so significantly lower the suggested sample size, and more presses can be used to describe the population.